In [2]:
import os
import yaml
import git
from cgi import test
from dataclasses import dataclass
from google.oauth2 import service_account
from snowflake.sqlalchemy import URL


path = os.path.expanduser('~')

def get_git_root(path):
    
        git_repo = git.Repo(path, search_parent_directories=True)
        git_root = git_repo.git.rev_parse("--show-toplevel")
        return (git_root)
    
##def return_git_path():
    
git_path = get_git_root(os.getcwd())


## profile vars
@dataclass
class IdentifyConfigVariables(Common):
    
    path_source: str
    profile_path: str
    project_path: str
    full_path: str

def assign_droughty_paths():

    if Common.env_vars == None:

        IdentifyConfigVariables.path_source = 'local_vars'

    elif Common.env_vars == 'enabled':

        IdentifyConfigVariables.path_source = 'env_vars'

        print ("Using environment variables")

    try:

        IdentifyConfigVariables.profile_path = Common.profile_dir

        print ("Using optional profile path")

        print (Common.profile_dir)

    except:

        path = os.path.expanduser('~')

        IdentifyConfigVariables.profile_pass = os.path.join(path,".droughty/profile.yaml")       

        print ("Using default profile path")

    if IdentifyConfigVariables.profile_path == None:

        raise Exception ("It looks like you don't have a profile file or you have directly specified the incorrect directory")  

paths = assign_droughty_paths()

def load_droughty_profile():

    IdentifyConfigVariables.project_path = Common.project_dir  

    with open(IdentifyConfigVariables.profile_path) as f:

        droughty_profile = yaml.load(f, Loader=yaml.FullLoader)

    return droughty_profile

def assign_droughty_project_paths():

    try:

        IdentifyConfigVariables.project_path = Common.project_dir  

        IdentifyConfigVariables.path_source = 'local_vars'

        print ("Using optional project path")

        print (Common.project_dir)


    except:

        IdentifyConfigVariables.project_path = Common.project_dir  

        filename = 'droughty_project.yaml'

        droughty_project = os.path.join(git_path,filename)

    if IdentifyConfigVariables.project_path == None:

        raise Exception ("It looks like you don't have a droughty_project file or you have directly specified the incorrect directory")  

project_path = assign_droughty_project_paths()


def load_droughty_project():

    IdentifyConfigVariables.project_path = Common.project_dir  

    with open(IdentifyConfigVariables.project_path) as f:
        droughty_project = yaml.load(f, Loader=yaml.FullLoader)

    return droughty_project
    
droughty_profile = load_droughty_profile()
droughty_project = load_droughty_project()

usage: ipykernel_launcher.py [-h] [--profile-dir PROFILE_DIR]
                             [--project-dir PROJECT_DIR] [--env-vars ENV_VARS]
                             {lookml,cube,dbml,dbt} ...
ipykernel_launcher.py: error: argument command: invalid choice: '/Users/lewischarlesbaker/Library/Jupyter/runtime/kernel-9c9b9204-d693-4b9d-b239-76810f3db2a3.json' (choose from 'lookml', 'cube', 'dbml', 'dbt')


AssertionError: 